# Unix File System and Search Functionality 

1. Basic structure of a unix file system ( Files and folders starting from root folder )
2. File class : Name, extension, is_directory (boolean), children (List[File]), date_created, File size, owner (str), file_permissions (Dict)
3. File System class: Name, root node, filters list [add filters to the file system]
4. Methods for filter : add_filter, add_file, AND_SEARCH and OR_SEARCH
5. Implement methods to search for files across the file system ( name, extension, file size and date created)
6. Base filter class (ABC)
7. Through inheritance, i'll create the specific filter classes (NameFilter, ExtensionFilter, DateFilter, FileSizeFilter)

In [23]:
class File():
    def __init__(self, file_name: str, file_size: int = 0, owner: str = "admin"):
        file_split = file_name.split('.')
        self.file_name = file_split[0]
        self.file_extension = file_split[1] if len(file_split) > 1 else ""
        self.owner = owner 
        self.permissions = {
            'read': True, 
            'write': True, 
            'execute': False
        }
        self.is_directory = False
        self.children = {}  # Can be either list or dict 
        if not self.file_extension:
            self.is_directory = True
        else:
            self.file_size = file_size
        self.created_date = datetime.datetime.now()

In [68]:
from enum import Enum
class FileType(Enum):
    """Enum for file types."""
    TEXT = ".txt"  
    IMAGE = '.jpg'
    DIRECTORY = '.dir'
    PDF = '.pdf'
    AUDIO = '.mp3'
    VIDEO = '.mp4'
    ARCHIVE = '.zip'



In [57]:
from abc import ABC, abstractmethod

class Filter(ABC):
    """
    Abstract base class for Filters
    """
    @abstractmethod
    def apply(self, root: File) -> bool:
        """
        Apply filter to the file
        """
        pass 
    
    def change_filter(self) -> bool:
        """
        Change filter value
        """
        pass

In [58]:
# Name Filter class 
class NameFilter(Filter):
    def __init__(self, name:str):
        self.name = name
    
    def apply(self, root:File) -> bool:
        if root.file_name == self.name:
            return True 
        return False
        

In [59]:
import datetime
# Date Filter class
class DateFilter(Filter):
    def __init__(self, from_date : datetime, to_date : datetime):
        self.from_date = from_date
        self.to_date = to_date
    
    def apply(self, root:File) -> File:
        if self.from_date <= self.created_date <= self.to_date:
            return True
        return False

In [60]:
import datetime
# File Size Filter class
class FileSizeFilter(Filter):
    def __init__(self, size: int, greater_than: bool):
        self.size = size
        self.greater_than = greater_than
    
    def apply(self, root:File) -> bool:
        if root.is_directory:
            return False
            
        if self.greater_than:
            if root.file_size > self.size:
                return True
        else:
            if root.file_size < self.size:
                return True
        return False

In [61]:
# File Extension Filter 

class FileExtensionFilter(Filter):
    def __init__(self, extension:str):
        self.extension = extension

    def apply(self, root:File) -> bool:
        if root.is_directory:
            return False
        if self.extension == root.file_extension:
            return True 
        return False

In [77]:
from typing import List, Literal
class FileSystem():
    """
    File System class that creates a heirarchial Unix like file system 

    name : name of the file system 
    root : root File class obj of the file system
    filters :  a list of Filter Class objects that will be applied to the Unix File Search methods
    """
    def __init__(self, file_system_name : str):
        self.name = file_system_name
        self.root = File('root')
        self.filters = []

    def add_file(self, file_name: str, file_size: int, owner: str):

        path_parts = file_name.split('/')

        if path_parts[0] != "root":
            raise Exception("File path must start with /root")

        current_dir = self.root

        for i in range(1, len(path_parts)-1):
            dir_path = path_parts[i]

            child_dir = None
            
            if dir_path in current_dir.children:
                child_dir = current_dir.children[dir_path]
            else:
                child_dir = File(dir_path)
                current_dir.children[dir_path] = child_dir

            current_dir = child_dir

        file_name = path_parts[-1]
        new_file = File(file_name, file_size, owner)
        current_dir.children[file_name] = new_file

        return new_file

    def remove_file(self, file_name: str):
        path_parts = file_name.split('/')
        
        if not path_parts[0] == "root":
            raise Exception("File path must start with /root")

        current_dir = self.root

        for i in range(1, len(path_parts)-1):
            dir_name = path_parts[i]

            if dir_name in current_dir.children:
                child_dir = current_dir.children[dir_name]
            else:
                raise Exception("File path mentioned doesn't exist in the File System.")

            current_dir = child_dir

        file_name = path_parts[-1]

        if file_name not in current_dir.children:
            raise Exception("File path mentioned doesn't exist in the File System.")
        else:
            del current_dir.children[file_name]  

        return True
        
    def add_filters(self, filter_obj:Filter):
        if not isinstance(filter_obj, Filter):
            raise TypeError("Filter must be an instance of the Filter class or its subclasses")

        self.filters.append(filter_obj)
        return True

    def apply_filters(self, file: File, logic: str = "AND") -> bool:
        if logic == 'OR':
            for filter in self.filters:
                if filter.apply(file):
                    return True
            return False
            
        elif logic == 'AND':
            for filter in self.filters:
                if not filter.apply(file):
                    return False
            return True 

        else:
            raise Exception('Conditonal logic not present in the function')

    def search(self, logic: str = 'AND') -> List[str]:
        results = []
        self.search_recursive(self.root, results,logic)
        return results
        
    def search_recursive(self, root: File, results : List[str], logic: str):
        
        if not root.is_directory:
            if self.apply_filters(root, logic):
                results.append(root.file_name + '.' + root.file_extension)

        else:
            for child_path, child_file in root.children.items():
                self.search_recursive(child_file, results, logic)

    def print_file_system(self):
        
        self.print_file_system_recursive(self.root, 0)

    def print_file_system_recursive(self, node: File, level: int):

        indent = '   '*level

        if node.is_directory:
            print(f"{indent}[DIR] {node.file_name}")
            for child_name, child in node.children.items():
                print_file_system(child, level + 1)
        else:
            print(f"{indent}[FILE] {node.file_name}.{node.file_extension} ({node.file_size} bytes, owned by {node.owner})")
        
        
            
        
        
    

In [78]:
import datetime

def test_file_system():
    # Create a file system
    fs = FileSystem("TestFS")
    
    # Test file creation
    print("Testing file creation...")
    fs.add_file("root/documents/report.txt", 1024, "user1")
    fs.add_file("root/documents/presentation.ppt", 2048, "user1")
    fs.add_file("root/images/photo.jpg", 3072, "user2")
    fs.add_file("root/images/logo.png", 512, "user2")
    fs.add_file("root/code/script.py", 256, "user3")
    
    # Print the file system structure
    print("\nFile system structure:")
    fs.print_file_system()
    
    # Test file removal
    print("\nTesting file removal...")
    fs.remove_file("root/documents/report.txt")
    print("File removed successfully!")
    
    # Print the updated file system structure
    print("\nUpdated file system structure:")
    fs.print_file_system()
    
    # Test filters
    print("\nTesting filters...")
    
    # Add filters
    txt_filter = FileExtensionFilter("txt")
    size_filter = FileSizeFilter(1000, True)
    date_filter = DateFilter(
        datetime.datetime.now() - datetime.timedelta(days=1),
        datetime.datetime.now() + datetime.timedelta(days=1)
    )
    name_filter = NameFilter("script")
    
    fs.add_filters(txt_filter)
    fs.add_filters(size_filter)
    
    # Search with OR logic
    print("\nSearching with OR logic:")
    results = fs.search("OR")
    print(f"Found {len(results)} files:")
    for file in results:
        print(f"- {file}")
    
    # Reset filters
    fs.filters = []
    
    # Add name filter
    fs.add_filters(name_filter)
    
    # Search with AND logic
    print("\nSearching with AND logic (name filter only):")
    results = fs.search("AND")
    print(f"Found {len(results)} files:")
    for file in results:
        print(f"- {file}")
    
    # Test error handling
    print("\nTesting error handling...")
    try:
        fs.add_file("documents/invalid.txt", 100, "user4")
    except Exception as e:
        print(f"Error caught successfully: {e}")
    
    try:
        fs.remove_file("root/nonexistent/file.txt")
    except Exception as e:
        print(f"Error caught successfully: {e}")
    
    print("\nAll tests completed!")




In [79]:
test_file_system()

Testing file creation...

File system structure:
[DIR] root
  [DIR] documents
    [FILE] report.txt (1024 bytes, owned by user1)
    [FILE] presentation.ppt (2048 bytes, owned by user1)
  [DIR] images
    [FILE] photo.jpg (3072 bytes, owned by user2)
    [FILE] logo.png (512 bytes, owned by user2)
  [DIR] code
    [FILE] script.py (256 bytes, owned by user3)

Testing file removal...
File removed successfully!

Updated file system structure:
[DIR] root
  [DIR] documents
    [FILE] presentation.ppt (2048 bytes, owned by user1)
  [DIR] images
    [FILE] photo.jpg (3072 bytes, owned by user2)
    [FILE] logo.png (512 bytes, owned by user2)
  [DIR] code
    [FILE] script.py (256 bytes, owned by user3)

Testing filters...

Searching with OR logic:
Found 2 files:
- presentation.ppt
- photo.jpg

Searching with AND logic (name filter only):
Found 1 files:
- script.py

Testing error handling...
Error caught successfully: File path must start with /root
Error caught successfully: File path mentio